# A* Extension
This notebook provides an interactive interface to visualize the A* extension for a 2-DoF environment. It allows you to select different benchmarks and adjust the parameters.

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from AStarExtension.evaluation.two_dof.IPTestSuite_2DoF import benchList
from AStarExtension.evaluation.two_dof.PlotEnvironments import visualizeBenchmark

# TODO: First, create an A star config and adapt it based on sliders

slider_layout = widgets.Layout(width='400px')
slider_style = {'description_width': '150px'}

benchmark_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(benchList)-1,
    step=1,
    description='Benchmark:',
    layout=slider_layout,
    style=slider_style
)

discretizationX_slider = widgets.SelectionSlider(
    options=[10, 20, 50, 100, 200],
    value=10,
    description='X Discretization:',
    layout=slider_layout,
    style=slider_style
)

discretizationY_slider = widgets.SelectionSlider(
    options=[10, 20, 50, 100, 200],
    value=10,
    description='Y Discretization:',
    layout=slider_layout,
    style=slider_style
)

reopening_slider = widgets.SelectionSlider(
    options=[False, True],
    value=False,
    description='Reopening:',
    layout=slider_layout,
    style=slider_style
)

lineCollision_slider = widgets.SelectionSlider(
    options=[False, True],
    value=False,
    description='Line Collision:',
    layout=slider_layout,
    style=slider_style
)

# Create output widget to capture the plots
output = widgets.Output()

# Function to update plot - gets current values from all sliders
def update_plot():
    with output:
        output.clear_output(wait=True)
        fig, ax = visualizeBenchmark(
            benchList[benchmark_slider.value],
            discretizationX=discretizationX_slider.value,
            discretizationY=discretizationY_slider.value
        )
        plt.show()

# Initial plot
update_plot()

# Single change handler for all sliders
def on_any_slider_change(change):
    update_plot()

# Attach the same handler to all three sliders
benchmark_slider.observe(on_any_slider_change, names='value')
discretizationX_slider.observe(on_any_slider_change, names='value')
discretizationY_slider.observe(on_any_slider_change, names='value')

# Display all sliders and output
display(benchmark_slider, discretizationX_slider, discretizationY_slider, lineCollision_slider, reopening_slider, output)



In [ ]:
# todo: perform A* search with the current parameters and visualize the result, as well as the stepwise visu